In [ ]:
addprocs(20)
using GynC
using JLD
using KernelDensity
using Plots; gr()

In [2]:
import Distributions: pdf, rand

In [3]:
type MeasErr <: Distribution{Multivariate, Continuous}
    rhos
end

function pdf(m::MeasErr, delta::Matrix)
  return errpdf(delta)
end

function rand(m::MeasErr)
  [rand(m.rhos[j]) for i=1:31, j=1:4]
end

function errpdf(delta)
    exp(-sumabs2(filter(x->!isnan(x), (delta ./ GynC.model_measerrors' ))))
end

#err = MeasErr([Normal(std) for std in GynC.model_measerrors]);

In [ ]:
@everywhere begin

import Distributions: pdf, rand
type MatrixNormalCentered <: Distribution
    sigmas
end

function rand(n::MatrixNormalCentered)
    map(s->rand(Normal(0,s)), n.sigmas)
end

function pdf(n::MatrixNormalCentered, x::Matrix)
    d = 0.
    for i in length(x)
        isnan(x[i]) && continue
        d += logpdf(Normal(0, n.sigmas[i]), x[i])
    end
    exp(d)
end

end

In [5]:
sigma = 0.1
err = MatrixNormalCentered(repmat(sigma*GynC.model_measerrors', 31))

MatrixNormalCentered(
sigmas: [1.2 0.1 4.0 0.15; 1.2 0.1 4.0 0.15; … ; 1.2 0.1 4.0 0.15; 1.2 0.1 4.0 0.15]
)


In [6]:
@time s=JLD.load("../data/0911/allsamples.jld")["samples"];

 18.461267 seconds (2.65 M allocations: 7.885 GB, 38.13% gc time)


In [7]:
function subsample(samplings::Vector{Matrix{Float64}}, n::Int, burnin::Int)
    res = Vector{Vector{Float64}}()
    nsamplings = length(samplings)
    for sampling in samplings
        nsamples = size(sampling, 1)
        step = floor(Int,(nsamples - burnin) / n * nsamplings)
        for i = burnin+1:step:nsamples
            s = sampling[i,:]
            push!(res, sampling[i,:])
        end
    end
    res
end

xs = subsample(s, 1000, 100_000);
@show length(xs);

  0.075954 seconds (36.85 k allocations: 3.572 MB)
length(xs) = 1035


In [8]:
function alldatas(; minmeas=10)
  datas = vcat([Lausanne(i).data for i=1:40])#, [GynC.Pfizer(i).data for i=1:13])
  datas = filter(d->length(d) - sum(isnan(d)) > 20, datas)
end

datas = alldatas();
@show length(datas);

36-element Array{Array{Float64,2},1}:
 [NaN NaN NaN NaN; NaN NaN NaN NaN; … ; NaN NaN NaN NaN; NaN NaN NaN NaN]               
 [4.07 5.04 74.3667 2.43; NaN NaN NaN NaN; … ; 4.31 2.82 130.21 8.91; NaN NaN NaN NaN]  
 [NaN NaN NaN NaN; NaN NaN NaN NaN; … ; NaN NaN NaN NaN; 4.19 4.72 57.4775 2.89]        
 [NaN NaN NaN NaN; NaN NaN NaN NaN; … ; NaN NaN NaN NaN; NaN NaN NaN NaN]               
 [NaN NaN NaN NaN; 5.75 6.1 20.8935 0.977; … ; NaN NaN NaN NaN; NaN NaN NaN NaN]        
 [NaN NaN NaN NaN; NaN NaN NaN NaN; … ; 14.1 4.8 103.514 0.7; NaN NaN NaN NaN]          
 [5.42 NaN 32.1438 1.51; NaN NaN NaN NaN; … ; 8.11 NaN 123.672 8.77; NaN NaN NaN NaN]   
 [NaN NaN NaN NaN; NaN NaN NaN NaN; … ; 8.54 5.25 61.5636 2.38; NaN NaN NaN NaN]        
 [NaN NaN NaN NaN; NaN NaN NaN NaN; … ; 4.6 6.56 39.7712 0.62; NaN NaN NaN NaN]         
 [NaN NaN NaN NaN; NaN NaN NaN NaN; … ; NaN NaN NaN NaN; NaN NaN NaN NaN]               
 [7.11 3.45 32.0076 1.44; NaN NaN NaN NaN; … ; NaN NaN NaN NaN; NaN NaN 

In [9]:
phi(x) = GynC.forwardsol(x)[:,GynC.measuredinds]
@time ys = phi.(xs);

naninds = find(x->any(isnan(x)), ys)
nonnaninds = deleteat!(collect(1:length(ys)), naninds)

xs = xs[nonnaninds]
ys = ys[nonnaninds];

  6.799754 seconds (28.12 M allocations: 589.086 MB, 4.53% gc time)


In [10]:
zmult = 100
zs = map(y->y+rand(err), repmat(ys, zmult));
@show length(zs)

length(zs) = 102900


102900

In [ ]:
m = GynC.LikelihoodModel(xs, ys, zs, datas, err);



sigmas = [2*KernelDensity.default_bandwidth(filter(x->!isnan(x),[d[i,j] for d in datas])) for i=1:31, j=1:4]
smoothkernel = MatrixNormalCentered(sigmas)

ms = GynC.smoothdata(m, 200, smoothkernel);

sigmanew = sqrt.(m.measerr.sigma .^ 2 + smoothkernel.sigmas .^ 2)
ms.measerr = MatrixNormalCentered(sigmanew)

In [ ]:
niter = 20
h = 0.5

wuni = ones(length(xs)) / length(xs);
w0 = wuni

ws = Dict()
@time ws["NPMLE"] = GynC.em(m, w0, niter)
@time ws["DS-MLE"] = GynC.em(ms, w0, niter);
@time ws["MPLE"]  = GynC.mple(m, w0, niter, 0.5, h)
@time ws["Reference Prior"] = GynC.mple(m, w0, niter, 1, h);

  0.024973 seconds (4.25 k allocations: 349.858 KB)


INFO: computing likelihood matrix (1029x7200)


  6.518672 seconds (1.31 M allocations: 178.617 MB, 1.82% gc time)


INFO: computing likelihood matrix (102900x1029)
INFO: computing likelihood matrix (36x1029)


In [13]:
gr()

Plots.GRBackend()

In [14]:
itercolors(n) = collect(colormap("blues", n))'

function plotkdeiters(xs, ws; bandwidthmult=0.1, kwargs...)
  #colors = itercolors(length(ws))
    
  plot(legend=false; kwargs...)
    
  for i in 1:length(ws)
    bw = KernelDensity.default_bandwidth(xs)
    k=kde(xs, weights=ws[i], bandwidth=bw*bandwidthmult)
    plot!(k.x, k.density)#, linecolor = colors[i])
  end
    
  plot!()
end

#nicespecies = [31 44 50 76]

spec = 44
for (k,v) in ws
    v=[v[end]]
    plotkdeiters(map(x->x[spec], xs), v; title="$spec $(GynC.samplednames[spec])  $k", ylims = (0,3.5)) |> display

    #plot(plot(map(w->GynC.hz(m,w), v), title="hz $k"), plot(map(w->GynC.logl(m,w), v), title="logl $k")) |> display
end

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 2.0 
 
 
 2.5 
 
 
 3.0 
 
 
 3.5 
 
 
 44 transition rate constant from Sc1 to Sc2 NPMLE 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 34.5407,381.014 34.815,381.014 35.0893,381.014 35.3636,381.015 35.6379,381.015 35.9123,381.015 36.1866,381.015 36.4609,381.015 36.7352,381.015 37.0095,381.015 
 37.2838,381.015 37.5581,381.015 37.8325,381.015 38.1068,381.014 38.3811,381.014 38.6554,381.014 38.9297,381.014 39.204,381.014 39.4783,381.014 39.7527,381.013 
 40.027,381.013 40.3013,381.013 40.5756,381.012 40.8499,381.012 41.1242,381.011 41.3986,381.011 41.6729,381.01 41.9472,381.009 42.2215,381.008 42.4958,381.007 
 42.7701,381.006 43.0444,381.005 43.3188,381.004 43.5931,381.002 43.8674,381.001 44.1417,381 44.416,380.998 44.6903,380.997 44.9646,380.996 45.239,380.994 
 45.5133,380.993 45.7876,380.992 46.0619,380.991 46.3362,380.99 46.6105,380.989 46.8848,380.988 47.1592,380.987 47.4335,380.987 47.7078,380.987 47.9821,380.986 
 48.2564,380.986 48.5307,380.986 48.805,380.986 49.0794,380.985 49.3537,380.985 49.628,380.984 49.9023,380.982 50.1766,380.98 50.4509,380.976 50.7253,380.972 
 50.9996,380.965 51.2739,380.957 51.5482,380.945 51.8225,380.931 52.0968,380.913 52.3711,380.889 52.6455,380.861 52.9198,380.825 53.1941,380.782 53.4684,380.73 
 53.7427,380.668 54.017,380.594 54.2913,380.508 54.5657,380.408 54.84,380.292 55.1143,380.159 55.3886,380.008 55.6629,379.839 55.9372,379.65 56.2115,379.442 
 56.4859,379.213 56.7602,378.964 57.0345,378.696 57.3088,378.41 57.5831,378.108 57.8574,377.791 58.1318,377.461 58.4061,377.122 58.6804,376.775 58.9547,376.425 
 59.229,376.073 59.5033,375.723 59.7776,375.377 60.052,375.038 60.3263,374.706 60.6006,374.384 60.8749,374.07 61.1492,373.763 61.4235,373.462 61.6978,373.164 
 61.9722,372.864 62.2465,372.557 62.5208,372.236 62.7951,371.893 63.0694,371.522 63.3437,371.112 63.618,370.655 63.8924,370.139 64.1667,369.557 64.441,368.897 
 64.7153,368.15 64.9896,367.306 65.2639,366.358 65.5383,365.298 65.8126,364.118 66.0869,362.814 66.3612,361.379 66.6355,359.812 66.9098,358.111 67.1841,356.275 
 67.4585,354.306 67.7328,352.207 68.0071,349.983 68.2814,347.642 68.5557,345.193 68.83,342.647 69.1043,340.02 69.3787,337.326 69.653,334.586 69.9273,331.82 
 70.2016,329.051 70.4759,326.304 70.7502,323.606 71.0245,320.984 71.2989,318.467 71.5732,316.081 71.8475,313.855 72.1218,311.813 72.3961,309.978 72.6704,308.371 
 72.9447,307.007 73.2191,305.899 73.4934,305.054 73.7677,304.475 74.042,304.158 74.3163,304.095 74.5906,304.274 74.865,304.675 75.1393,305.279 75.4136,306.06 
 75.6879,306.99 75.9622,308.04 76.2365,309.18 76.5108,310.38 76.7852,311.612 77.0595,312.848 77.3338,314.063 77.6081,315.235 77.8824,316.347 78.1567,317.383 
 78.431,318.334 78.7054,319.191 78.9797,319.954 79.254,320.621 79.5283,321.197 79.8026,321.689 80.0769,322.107 80.3512,322.46 80.6256,322.762 80.8999,323.027 
 81.1742,323.267 81.4485,323.496 81.7228,323.726 81.9971,323.97 82.2715,324.236 82.5458,324.532 82.8201,324.864 83.0944,325.233 83.3687,325.64 83.643,326.081 
 83.9173,326.55 84.1917,327.039 84.466,327.535 84.7403,328.026 85.0146,328.497 85.2889,328.931 85.5632,329.312 85.8375,329.622 86.1119,329.846 86.3862,329.969 
 86.6605,329.979 86.9348,329.865 87.2091,329.623 87.4834,329.248 87.7577,328.742 88.0321,328.111 88.3064,327.363 88.5807,326.512 88.855,325.575 89.1293,324.573 
 89.4036,323.527 89.6779,322.463 89.9523,321.406 90.2266,320.382 90.5009,319.416 90.7752,318.533 91.0495,317.752 91.3238,317.093 91.5982,316.57 91.8725,316.193 
 92.1468,315.969 92.4211,315.899 92.6954,315.981 92.9697,316.205 93.244,316.563 93.5184,317.038 93.7927,317.615 94.067,318.272 94.3413,318.989 94.6156,319.746 
 94.8899,320.521 95.1642,321.293 95.4386,322.045 95.7129,322.76 95.9872,323.423 96.

In [15]:
plotkde!{T<:Real}(xs, w::Vector{T}) = plotkde!(xs, [w])

function plotkde!{T<:Real}(xs, ws::Vector{Vector{T}})
   ks = [kde(xs, weights=w) for w in ws]
   for k in ks
     plot!(k.x, k.density)
   end
end

    

w = wuni
plot()
pprior = plotkde!((x->x[8]).(xs), ws["NPMLE"])
plot!()

LoadError: LoadError: MethodError: no method matching plotkde!(::Array{Float64,1}, ::Array{Any,1})
Closest candidates are:
  plotkde!{T<:Real}(::Any, !Matched::Array{T<:Real,1}) at In[15]:1
  plotkde!{T<:Real}(::Any, !Matched::Array{Array{T<:Real,1},1}) at In[15]:4
while loading In[15], in expression starting on line 14

In [ ]:
ts = 0:1/5:30
species = 3

sols = [GynC.forwardsol(x, ts)[:,GynC.measuredinds[species]] for x in xs];

In [17]:
function plottrajdens(ts, sols, w)
  msol = hcat(sols...)
  bnd = (-1, quantile(vec(msol), 0.99) + 1)

    
  kdes = [KernelDensity.kde(msol[t,:], boundary = bnd, weights=w) for t in 1:size(msol, 1)]
  ys = kdes[1].x

  dens = hcat([k.density for k in kdes]...)
  clims = (0, quantile(vec(dens),0.98))
    
  contour(ts, ys, hcat([k.density for k in kdes]...), clims=clims, fill=true, seriescolor = :heat, legend=true)
end

plottrajdens(ts, sols, wuni)

LoadError: LoadError: InexactError()
while loading In[17], in expression starting on line 15

In [18]:
function plottraj(ts, sols, w; yquantile = 0.99, alphamult = 10)
  ylims = (0, quantile(vcat(sols...), 0.99))
  plot(ts, sols, alpha=w'*alphamult, legend=false, ylims=ylims, color = :black, linewidth=3)
end

function plotdatas!(datas, species = 3)
  specdatas = map(d->d[:,species], datas)
  scatter!(0:30, specdatas, color=:orange, legend=false)
end

plottraj(ts, sols, wuni)
plotdatas!(datas)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 0 
 
 
 1000 
 
 
 2000 
 
 
 3000 
 
 
 4000 
 
<polyline clip-path="url(#clip02)" style="stroke:#00002d; stroke-width:2.4; stroke-opacity:0.00971817; fill:none" points="
 59.0701,372.313 62.5458,370.83 66.0215,370.944 69.4972,371.326 72.9729,371.742 76.4485,372.151 79.9242,372.545 83.3999,372.922 86.8756,373.283 90.3513,373.625 
 93.827,373.95 97.3026,374.256 100.778,374.543 104.254,374.81 107.73,375.059 111.205,375.289 114.681,375.499 118.157,375.692 121.632,375.866 125.108,376.022 
 128.584,376.16 132.059,376.283 135.535,376.388 139.011,376.473 142.487,376.541 145.962,376.59 149.438,376.627 152.914,376.655 156.389,376.663 159.865,376.65 
 163.341,376.625 166.816,376.582 170.292,376.528 173.768,376.463 177.243,376.381 180.719,376.281 184.195,376.172 187.67,376.053 191.146,375.912 194.622,375.776 
 198.097,375.629 201.573,375.466 205.049,375.295 208.524,375.121 212,374.94 215.476,374.752 218.952,374.556 222.427,374.354 225.903,374.145 229.379,373.933 
 232.854,373.71 236.33,373.475 239.806,373.236 243.281,372.979 246.757,372.725 250.233,372.448 253.708,372.169 257.184,371.864 260.66,371.542 264.135,371.198 
 267.611,370.832 271.087,370.432 274.562,369.996 278.038,369.515 281.514,368.979 284.989,368.37 288.465,367.67 291.941,366.842 295.417,365.843 298.892,364.607 
 302.368,363.036 305.844,360.998 309.319,358.312 312.795,354.809 316.271,350.425 319.746,345.502 323.222,341.163 326.698,339.037 330.173,340.156 333.649,344.002 
 337.125,349.037 340.6,353.948 344.076,358.133 347.552,361.475 351.027,364.048 354.503,365.953 357.979,367.295 361.454,368.161 364.93,368.63 368.406,368.778 
 371.882,368.681 375.357,368.402 378.833,367.992 382.309,367.496 385.784,366.947 389.26,366.373 392.736,365.786 396.211,365.191 399.687,364.593 403.163,364.007 
 406.638,363.454 410.114,362.952 413.59,362.505 417.065,362.105 420.541,361.723 424.017,361.323 427.492,360.867 430.968,360.322 434.444,359.673 437.919,358.931 
 441.395,358.135 444.871,357.358 448.347,356.703 451.822,356.301 455.298,356.299 458.774,356.848 462.249,358.091 465.725,360.15 469.201,362.153 472.676,362.835 
 476.152,363.278 479.628,363.693 483.103,364.114 486.579,364.546 490.055,364.986 493.53,365.432 497.006,365.881 500.482,366.33 503.957,366.777 507.433,367.219 
 510.909,367.654 514.384,368.081 517.86,368.498 521.336,368.903 524.812,369.296 528.287,369.675 531.763,370.041 535.239,370.393 538.714,370.731 542.19,371.057 
 545.666,371.372 549.141,371.676 552.617,371.97 556.093,372.254 559.568,372.528 563.044,372.791 566.52,373.041 569.995,373.276 573.471,373.492 576.947,373.69 
 580.422,373.869 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#00002d; stroke-width:2.4; stroke-opacity:0.00971817; fill:none" points="
 59.0701,368.682 62.5458,370.015 66.0215,371.141 69.4972,372.093 72.9729,372.907 76.4485,373.607 79.9242,374.211 83.3999,374.732 86.8756,375.179 90.3513,375.563 
 93.827,375.89 97.3026,376.166 100.778,376.397 104.254,376.584 107.73,376.731 111.205,376.845 114.681,376.928 118.157,376.983 121.632,377.012 125.108,377.018 
 128.584,377.01 132.059,376.983 135.535,376.941 139.011,376.891 142.487,376.845 145.962,376.762 149.438,376.689 152.914,376.606 156.389,376.521 159.865,376.434 
 163.341,376.337 166.816,376.233 170.292,376.126 173.768,376.01 177.243,375.892 180.719,375.771 184.195,375.641 187.67,375.51 191.146,375.373 194.622,375.233 
 198.097,375.086 201.573,374.935 205.049,374.789 208.524,374.61 212,374.453 215.476,374.288 218.952,374.12 222.427,373.936 225.903,373.741 229.379,373.555 
 232.854,373.355 236.33,373.148 239.806,372.932 243.281,372.704 246.757,372.464 250.233,372.209 253.708,371.939 257.184,371.651 260.66,371.34 264.135,371.004 
 267.611,370.637 271.087,370.233 274.562,369.784 278.038,369.28 281.514,368.706 284.989,368.043 288.465,367.266 291.941,366.332 295.417,365.186 298.892,363.765 
 302.368,361.91

In [19]:
function bayesposterior(m, data, wprior)
    L = likelihoodmat(m.ys, data, m.measerr)
    GynC.emiteration(wprior, L)
end

patientdata = [datas[3]]

plottraj(ts, sols, bayesposterior(m, patientdata, wuni))
plotdatas!(patientdata)

INFO: computing likelihood matrix (1029x1)


LoadError: LoadError: UndefVarError: plotdatas not defined
while loading In[19], in expression starting on line 9

In [20]:
plottrajdens(ts, sols, bayesposterior(m, [datas[patient]], wuni))
plotdatas!(patientdata)

LoadError: LoadError: UndefVarError: patient not defined
while loading In[20], in expression starting on line 1